In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
conf=SparkConf().setAppName("BDProject").setMaster("local[3]")

In [4]:
sc=SparkContext(conf=conf)

In [5]:
rawBadges=sc.textFile("travel.stackexchange.com\Badges.xml")

In [6]:
rawUsers=sc.textFile("travel.stackexchange.com\\Users.xml")

In [7]:
rawUsers.take(3)

['<?xml version="1.0" encoding="utf-8"?>',
 '<users>',
 '  <row Id="-1" Reputation="1" CreationDate="2011-06-21T15:16:44.253" DisplayName="Community" LastAccessDate="2011-06-21T15:16:44.253" WebsiteUrl="http://meta.stackexchange.com/" Location="on the server farm" AboutMe="&lt;p&gt;Hi, I\'m not really a person.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I\'m a background process that helps keep this site clean!&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I do things like&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;Randomly poke old unanswered questions every hour so they get some attention&lt;/li&gt;&#xA;&lt;li&gt;Own community questions and answers so nobody gets unnecessary reputation from them&lt;/li&gt;&#xA;&lt;li&gt;Own downvotes on spam/evil posts that get permanently deleted&lt;/li&gt;&#xA;&lt;li&gt;Own suggested edits from anonymous users&lt;/li&gt;&#xA;&lt;li&gt;&lt;a href=&quot;http://meta.stackexchange.com/a/92006&quot;&gt;Remove abandoned questions&lt;/a&gt;&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;" Views="0" UpV

In [8]:
def cleaningRDD(RDDIn):
    def cleaning(listIn):
        listOut=[]
        for x in listIn:
            if listIn.index(x)%2==0:
                listOut.append(x.replace(' ','').replace('<','').replace('=',''))
            else:
                listOut.append(x)
        return {listOut[i]:listOut[i+1] for i in range(0,len(listOut)-1,2)}
    return RDDIn.filter(lambda x: x.startswith('  <row Id=')).\
map(lambda x: x.split('"')).map(cleaning)

In [9]:
badges=cleaningRDD(rawBadges)

In [10]:
badges.take(3)

[{'Class': '3',
  'Date': '2011-06-21T20:16:48.910',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '2',
  'rowId': '1'},
 {'Class': '3',
  'Date': '2011-06-21T20:16:48.910',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '3',
  'rowId': '2'},
 {'Class': '3',
  'Date': '2011-06-21T20:16:48.910',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '4',
  'rowId': '3'}]

In [11]:
badges.map(lambda x: x['Date']).take(10)

['2011-06-21T20:16:48.910',
 '2011-06-21T20:16:48.910',
 '2011-06-21T20:16:48.910',
 '2011-06-21T20:21:49.157',
 '2011-06-21T20:21:49.157',
 '2011-06-21T20:21:49.157',
 '2011-06-21T20:21:49.173',
 '2011-06-21T20:21:49.173',
 '2011-06-21T20:21:49.173',
 '2011-06-21T20:21:49.173']

In [12]:
slownik={'Class': '3','Date': '2011-06-21T20:16:48.910','Name': 'Autobiographer','TagBased': 'False','UserId': '2','rowId': '1'}

In [13]:
slownik.keys()

dict_keys(['Class', 'Date', 'Name', 'TagBased', 'UserId', 'rowId'])

In [14]:
x=slownik['Date']
x

'2011-06-21T20:16:48.910'

In [15]:
users=cleaningRDD(rawUsers)

In [16]:
users.take(1)

[{'AboutMe': "&lt;p&gt;Hi, I'm not really a person.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I'm a background process that helps keep this site clean!&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I do things like&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;Randomly poke old unanswered questions every hour so they get some attention&lt;/li&gt;&#xA;&lt;li&gt;Own community questions and answers so nobody gets unnecessary reputation from them&lt;/li&gt;&#xA;&lt;li&gt;Own downvotes on spam/evil posts that get permanently deleted&lt;/li&gt;&#xA;&lt;li&gt;Own suggested edits from anonymous users&lt;/li&gt;&#xA;&lt;li&gt;&lt;a href=&quot;http://meta.stackexchange.com/a/92006&quot;&gt;Remove abandoned questions&lt;/a&gt;&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;",
  'AccountId': '-1',
  'CreationDate': '2011-06-21T15:16:44.253',
  'DisplayName': 'Community',
  'DownVotes': '14237',
  'LastAccessDate': '2011-06-21T15:16:44.253',
  'Location': 'on the server farm',
  'Reputation': '1',
  'UpVotes': '3147',
  'Views': '0',
  'WebsiteUrl

In [17]:
rawVotes=sc.textFile("travel.stackexchange.com\Votes.xml")
votes=cleaningRDD(rawVotes)

In [18]:
votes.take(5)

[{'CreationDate': '2011-06-21T00:00:00.000',
  'PostId': '1',
  'VoteTypeId': '2',
  'rowId': '1'},
 {'CreationDate': '2011-06-21T00:00:00.000',
  'PostId': '1',
  'VoteTypeId': '2',
  'rowId': '2'},
 {'CreationDate': '2011-06-21T00:00:00.000',
  'PostId': '2',
  'VoteTypeId': '2',
  'rowId': '3'},
 {'CreationDate': '2011-06-21T00:00:00.000',
  'PostId': '3',
  'VoteTypeId': '2',
  'rowId': '4'},
 {'CreationDate': '2011-06-21T00:00:00.000',
  'PostId': '5',
  'UserId': '13',
  'VoteTypeId': '5',
  'rowId': '5'}]

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [20]:
spark = SparkSession.builder \
    .appName('DataFrame_1') \
    .master('local[*]') \
    .getOrCreate()

In [21]:
usersDF=users.map(lambda x: Row(**x)).toDF()

In [22]:
usersDF.printSchema()

root
 |-- AboutMe: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- DownVotes: string (nullable = true)
 |-- LastAccessDate: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Reputation: string (nullable = true)
 |-- UpVotes: string (nullable = true)
 |-- Views: string (nullable = true)
 |-- WebsiteUrl: string (nullable = true)
 |-- rowId: string (nullable = true)



Tu jest trick- trzeba schemat napisać jakąś funkcją, bo jak się okazuje schemat DF nie tworzy się na podstawie najdłuższego wiersza - podchwytliwe

In [23]:
usersDF.show(4)

Py4JJavaError: An error occurred while calling o109.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 6, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 12 fields are required while 13 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$.fromJava(EvaluatePython.scala:138)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:235)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:228)
	at org.apache.spark.rdd.RDD$$anon

In [24]:
votesDF=votes.toDF()

C:\apache-spark\spark-2.2.0-bin-hadoop2\spark-2.2.0-bin-hadoop2.7\python\pyspark\sql\session.py:351: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [27]:
votesDF.show()

+--------------------+------+----------+-----+
|        CreationDate|PostId|VoteTypeId|rowId|
+--------------------+------+----------+-----+
|2011-06-21T00:00:...|     1|         2|    1|
|2011-06-21T00:00:...|     1|         2|    2|
|2011-06-21T00:00:...|     2|         2|    3|
|2011-06-21T00:00:...|     3|         2|    4|
|2011-06-21T00:00:...|     5|         5|    5|
|2011-06-21T00:00:...|     2|         5|    6|
|2011-06-21T00:00:...|     7|         2|    7|
|2011-06-21T00:00:...|     6|         2|    8|
|2011-06-21T00:00:...|     1|         2|    9|
|2011-06-21T00:00:...|     7|         2|   10|
|2011-06-21T00:00:...|     2|         2|   11|
|2011-06-21T00:00:...|     3|         2|   12|
|2011-06-21T00:00:...|     3|         2|   13|
|2011-06-21T00:00:...|    11|         2|   14|
|2011-06-21T00:00:...|     2|         2|   15|
|2011-06-21T00:00:...|     9|         2|   16|
|2011-06-21T00:00:...|     3|         2|   17|
|2011-06-21T00:00:...|     2|         5|   18|
|2011-06-21T0